In [87]:
using CSV, DataFrames, Statistics, FreqTables, LinearAlgebra

In [88]:
a1=DataFrame(CSV.File("./csv/asts.csv"));

In [89]:
b1=DataFrame(CSV.File("./csv/philinst.csv")); # comparing to output of instructor.pl

In [90]:
c1=DataFrame(CSV.File("./csv/classlist.csv")); # loading UBC downloaded classlist

In [91]:
m1=DataFrame(CSV.File("./csv/marks.csv"));

In [92]:
a2=a1[a1.course .== 514,:]; # filter for course id 326

In [93]:
d1=Dict{String, Float64}() # associate ast with cont
for i in 1:length(a2[:,:ast])
    merge!(d1,Dict(a2[i,:ast]=>a2[i,:cont]))
end

In [94]:
m2=DataFrame() # filter for marks relevant to course id
dup=[]
for i in 1:length(a2[:,:ast])
    m2=vcat(m2,m1[m1.ast .== a2[i,:ast],:])
    dup=vcat(dup,sum(nonunique(m1[m1.ast .== a2[i,:ast],:],:sdc)))
end

In [95]:
dup # these should all be zero or else there is duplication

23-element Vector{Any}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [96]:
function ast2cont(x)
    d1[x]
end

ast2cont (generic function with 1 method)

In [97]:
m3=transform(m2,:ast => ByRow(ast2cont) => :cont);

In [98]:
m3g=groupby(m3,:sdc);

In [99]:
b2=combine(m3g,[:mark,:cont] => ((m,c) -> dot(m,c)) => :totalj);

In [100]:
# next steps: (1) account for missing data (2) replace sdc by name and sno (3) sort and export to csv

In [101]:
b3=innerjoin(b1,b2,on=:sdc,makeunique=true);

In [102]:
function rw(x)
    trunc(Int,x+0.5)
end

rw (generic function with 1 method)

In [103]:
b4=sort(select(b3,:sno,:sdc,:fn,:ln,:total,:totalj => ByRow(rw) => :juliatotal),:juliatotal,rev=true);

In [104]:
b5=select(b4,:sno,:juliatotal);

In [105]:
c2=rename!(c1,Symbol("Student Number") => :sno);

In [106]:
c3=innerjoin(c2,b5,on=:sno);

In [107]:
c4=select(c3,Not(:"Percent Grade"));

In [108]:
c5=select(c4,1:6,9,7,8);

In [109]:
c6=rename(c5,:sno => :"Student Number",:juliatotal => :"Percent Grade")

,Session,Campus,Student Number,Subject,Course,Section,Percent Grade,Standing
,String7,String3,Int64,String7,String7,Int64,Int64,Missing
1,2022S,UBC,38426516,PHIL,375A,1,92,missing
2,2022S,UBC,22213623,PHIL,375A,1,90,missing
3,2022S,UBC,81789018,PHIL,375A,1,90,missing
4,2022S,UBC,95078424,PHIL,375A,1,90,missing
5,2022S,UBC,77049898,PHIL,375A,1,90,missing
6,2022S,UBC,39908355,PHIL,375A,1,89,missing
7,2022S,UBC,40747941,PHIL,375A,1,89,missing
8,2022S,UBC,44818177,PHIL,375A,1,89,missing
9,2022S,UBC,30161814,PHIL,375A,1,88,missing


In [110]:
CSV.write("./csv/grades4sub.csv",c6)

"./csv/grades4sub.csv"